# Amazon SageMaker Workshop
### _**Evaluation**_

---
In this part of the workshop we will get the previous model we trained to Predict Mobile Customer Departure and evaluate its performance with a test dataset.

---

## Contents

1. [Background](#Background) - Getting the model trained in the previous lab.
2. [Evaluate](#Evaluate)
    * Creating a script to evaluate model
    * Using [SageMaker Processing](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html) jobs to automate evaluation of models
3. [Exercise](#a_Exercise) - customizing metrics and evaluation reports
4. [Wrap-up - end of Evaluation Lab](#Wrap-up)


---

## Background

In the previous [Modeling](../2-Modeling/modeling.ipynb) lab we used SageMaker trained models by creating multiple SageMaker training jobs.

Install and import some packages we'll need for this lab:

In [3]:
import boto3
import sagemaker
from sagemaker.s3 import S3Uploader, S3Downloader

In [4]:
sm_sess = sagemaker.session.Session()
role = sagemaker.get_execution_role()

Get the variables from initial setup:

In [5]:
%store -r bucket
%store -r prefix
%store -r region
%store -r docker_image_name

In [6]:
bucket, prefix, region, docker_image_name

('sagemaker-studio-us-east-2-309044321483',
 'xgboost-churn',
 'us-east-2',
 '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:1.3-1')

---
### - if you _**skipped**_ the data preparation lab follow instructions:

   - **run this [notebook](./config/pre_setup.ipynb)**
   - load the model S3 URI:

In [7]:
# # Uncomment if you skipped the data preparation lab

# %store -r s3uri_model
# !cp config/model.tar.gz ./

# %store -r s3uri_test

---
### - if you _**have done**_ the previous labs

Download the model and test data from S3:

In [8]:
# Uncomment if you have done the previous labs

# Get name of training job and other variables
%store -r training_job_name
training_job_name

'workshop-framework-xgboost-customer-chu-2021-06-10-00-06-45-384'

In [9]:
# Uncomment if you have done the previous labs
estimator = sagemaker.estimator.Estimator.attach(training_job_name)
s3uri_model = estimator.model_data
print("\ns3uri_model =",s3uri_model)

S3Downloader.download(s3uri_model, ".")


2021-06-10 00:16:15 Starting - Preparing the instances for training
2021-06-10 00:16:15 Downloading - Downloading input data
2021-06-10 00:16:15 Training - Training image download completed. Training in progress.
2021-06-10 00:16:15 Uploading - Uploading generated training model
2021-06-10 00:16:15 Completed - Training job completed

s3uri_model = s3://sagemaker-studio-us-east-2-309044321483/xgboost-churn/output/workshop-framework-xgboost-customer-chu-2021-06-10-00-06-45-384/output/model.tar.gz


In [10]:
%store -r s3uri_test
S3Downloader.download(s3uri_test, ".")

---

Now you should have the `model.tar.gz` file in the 3-Evaluation directory 

(click the refresh button)

![refresh_dir.png](./media/refresh_dir.png)

# Evaluate model

Let's create a simple evaluation with some Scikit-Learn Metrics like [Area Under the Curve (AUC)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html) and [Accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html).

In [11]:
import json
import os
import tarfile
import logging
import pickle

import pandas as pd
import xgboost

from sklearn.metrics import classification_report, roc_auc_score, accuracy_score


model_path = "model.tar.gz"
with tarfile.open(model_path) as tar:
    tar.extractall(path=".")

print("Loading xgboost model.")
model = pickle.load(open("xgboost-model", "rb"))
model

Loading xgboost model.


In [12]:
print("Loading test input data")
test_path = "config/test-dataset.csv"
df = pd.read_csv(test_path, header=None)
df

Loading test input data


,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0,186,0,137.8,97,187.7,118,146.4,85,8.7,...,0,0,0,0,0,1,1,0,1,0
1,0,132,25,113.2,96,269.9,107,229.1,87,7.1,...,0,0,0,0,1,0,1,0,0,1
2,0,112,17,183.2,95,252.8,125,156.7,95,9.7,...,0,0,0,0,1,0,1,0,0,1
3,0,91,24,93.5,112,183.4,128,240.7,133,9.9,...,0,0,0,0,0,1,0,1,0,1
4,0,22,0,110.3,107,166.5,93,202.3,96,9.5,...,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,0,59,0,166.3,95,239.3,87,123.2,108,10.0,...,0,0,0,0,0,1,1,0,1,0
330,0,127,14,143.2,99,169.9,91,221.6,77,11.6,...,0,0,0,0,1,0,1,0,0,1
331,0,86,0,166.2,112,255.3,81,228.1,97,5.4,...,0,0,0,0,1,0,1,0,1,0
332,0,36,43,29.9,123,129.1,117,325.9,105,8.6,...,0,0,0,1,0,0,1,0,0,1


In [13]:
print("Reading test data. We should get an `DMatrix` object...")
y_test = df.iloc[:, 0].to_numpy()
df.drop(df.columns[0], axis=1, inplace=True)
X_test = xgboost.DMatrix(df.values)
X_test

Reading test data. We should get an `DMatrix` object...


In [14]:
print("Performing predictions against test data.")
predictions_probs = model.predict(X_test)
predictions = predictions_probs.round()

print("Creating classification evaluation report")
acc = accuracy_score(y_test, predictions)
auc = roc_auc_score(y_test, predictions_probs)

print("Accuracy =", acc)
print("AUC =", auc)

Performing predictions against test data.
Creating classification evaluation report
Accuracy = 0.9550898203592815
AUC = 0.9146998834498835


### Creating a classification report

Now, let's save the results in a JSON file, following the structure defined in SageMaker docs:
https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html

We'll use this logic later in [Lab 6-Pipelines](../6-Pipelines/pipelines.ipynb):

In [15]:
import pprint
# The metrics reported can change based on the model used - check the link for the documentation 
report_dict = {
    "binary_classification_metrics": {
        "accuracy": {
            "value": acc,
            "standard_deviation": "NaN",
        },
        "auc": {"value": auc, "standard_deviation": "NaN"},
    },
}

print("Classification report:")
pprint.pprint(report_dict)

Classification report:
{'binary_classification_metrics': {'accuracy': {'standard_deviation': 'NaN',
                                                'value': 0.9550898203592815},
                                   'auc': {'standard_deviation': 'NaN',
                                           'value': 0.9146998834498835}}}


In [16]:
evaluation_output_path = os.path.join(
    ".", "evaluation.json"
)
print("Saving classification report to {}".format(evaluation_output_path))

with open(evaluation_output_path, "w") as f:
    f.write(json.dumps(report_dict))

Saving classification report to ./evaluation.json


---

## Ok, now we have working code. Let's put it in a Python Script

In [15]:
%%writefile evaluate.py
"""Evaluation script for measuring model accuracy."""

import json
import os
import tarfile
import logging
import pickle

import pandas as pd
import xgboost

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

# May need to import additional metrics depending on what you are measuring.
# See https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score

def get_dataset(dir_path, dataset_name) -> pd.DataFrame:
    files = [ os.path.join(dir_path, file) for file in os.listdir(dir_path) ]
    if len(files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(files, dataset_name))
    raw_data = [ pd.read_csv(file, header=None) for file in files ]
    df = pd.concat(raw_data)
    return df

if __name__ == "__main__":
    model_path = "/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path="..")

    logger.debug("Loading xgboost model.")
    model = pickle.load(open("xgboost-model", "rb"))

    logger.info("Loading test input data")
    test_path = "/opt/ml/processing/test"
    df = get_dataset(test_path, "test_set")

    logger.debug("Reading test data.")
    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)
    X_test = xgboost.DMatrix(df.values)

    logger.info("Performing predictions against test data.")
    predictions_probs = model.predict(X_test)
    predictions = predictions_probs.round()

    logger.info("Creating classification evaluation report")
    acc = accuracy_score(y_test, predictions)
    auc = roc_auc_score(y_test, predictions_probs)

    # The metrics reported can change based on the model used, but it must be a specific name per (https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html)
    report_dict = {
        "binary_classification_metrics": {
            "accuracy": {
                "value": acc,
                "standard_deviation": "NaN",
            },
            "auc": {"value": auc, "standard_deviation": "NaN"},
        },
    }

    logger.info("Classification report:\n{}".format(report_dict))

    evaluation_output_path = os.path.join(
        "/opt/ml/processing/evaluation", "evaluation.json"
    )
    logger.info("Saving classification report to {}".format(evaluation_output_path))

    with open(evaluation_output_path, "w") as f:
        f.write(json.dumps(report_dict))


Writing evaluate.py


---

## Ok, now we are finally running this script with a simple call to SageMaker Processing!

In [17]:
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)

In [17]:
# Processing step for evaluation
processor = ScriptProcessor(
    image_uri=docker_image_name,
    command=["python3"],
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name="CustomerChurn/eval-script",
    sagemaker_session=sm_sess,
    role=role,
)

In [18]:
entrypoint = "evaluate.py"

In [18]:
from time import strftime, gmtime
# Helper to create timestamps
create_date = lambda: strftime("%Y-%m-%d-%H-%M-%S", gmtime())

In [20]:
processor.run(
    code=entrypoint,
    inputs=[
        sagemaker.processing.ProcessingInput(
            source=s3uri_model,
            destination="/opt/ml/processing/model",
        ),
        sagemaker.processing.ProcessingInput(
            source=s3uri_test,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        sagemaker.processing.ProcessingOutput(
            output_name="evaluation", source="/opt/ml/processing/evaluation"
        ),
    ],
    job_name=f"CustomerChurnEval-{create_date()}"
)


Job Name:  CustomerChurnEval-2021-06-10-00-18-17
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-studio-us-east-2-309044321483/xgboost-churn/output/workshop-framework-xgboost-customer-chu-2021-06-10-00-06-45-384/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-studio-us-east-2-309044321483/xgboost-churn/data/test/test-dataset.csv', 'LocalPath': '/opt/ml/processing/test', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-309044321483/CustomerChurnEval-2021-06-10-00-18-17/input/code/evaluate.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Pref

If everything went well, the SageMaker Processing job must have created the JSON with the evaluation report of our model and saved it in S3.

In addition, under the hood, SageMaker Processing has uploaded our `evaluate.py` script to S3. Let's check where the script was saved:

In [21]:
for proc_in in processor.latest_job.inputs:
    if proc_in.input_name == "code":
        s3_evaluation_code_uri = proc_in.source 
        
s3_evaluation_code_uri

's3://sagemaker-us-east-2-309044321483/CustomerChurnEval-2021-06-10-00-18-17/input/code/evaluate.py'

#### Let's store the S3 URI where our evaluation script was saved for later

In [22]:
%store s3_evaluation_code_uri

Stored 's3_evaluation_code_uri' (str)


### Let's check it the evaluation report from S3!

In [23]:
out_s3_report_uri = processor.latest_job.outputs[0].destination
out_s3_report_uri

's3://sagemaker-us-east-2-309044321483/CustomerChurnEval-2021-06-10-00-18-17/output/evaluation'

In [24]:
reports_list = S3Downloader.list(out_s3_report_uri)
reports_list

['s3://sagemaker-us-east-2-309044321483/CustomerChurnEval-2021-06-10-00-18-17/output/evaluation/evaluation.json']

In [25]:
report = S3Downloader.read_file(reports_list[0])

print("=====Model Report====")
print(json.dumps(json.loads(report.split('\n')[0]), indent=2))

=====Model Report====
{
  "binary_classification_metrics": {
    "accuracy": {
      "value": 0.9550898203592815,
      "standard_deviation": "NaN"
    },
    "auc": {
      "value": 0.9146998834498835,
      "standard_deviation": "NaN"
    }
  }
}


# Wrap-up

Now that we finished the **evaluation lab**, let's make everything here re-usable. It may come in handy later (spoiler alert - when creating Pipelines)...

In [26]:
%%writefile ../6-Pipelines/my_labs_solutions/evaluation_solution.py
import sagemaker
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)

def get_evaluation_processor(docker_image_name) -> ScriptProcessor:
    
    role = sagemaker.get_execution_role()
    sm_sess = sagemaker.session.Session()

    # Processing step for evaluation
    processor = ScriptProcessor(
        image_uri=docker_image_name,
        command=["python3"],
        instance_type="ml.m5.xlarge",
        instance_count=1,
        base_job_name="CustomerChurn/eval-script",
        sagemaker_session=sm_sess,
        role=role,
    )
    
    return processor

Writing ../6-Pipelines/my_labs_solutions/evaluation_solution.py


---
# [Extra] Fairness and Explainability with SageMaker Clarify

## Overview
Amazon SageMaker Clarify helps improve your machine learning models by detecting potential bias and helping explain how these models make predictions. The fairness and explainability functionality provided by SageMaker Clarify takes a step towards enabling AWS customers to build trustworthy and understandable machine learning models. The product comes with the tools to help you with the following tasks.

* Measure biases that can occur during each stage of the ML lifecycle (data collection, model training and tuning, and monitoring of ML models deployed for inference).
* Generate model governance reports targeting risk and compliance teams and external regulators.
* Provide explanations of the data, models, and monitoring used to assess predictions.


### - if you _**have done**_ the previous labs


In [19]:
session = sagemaker.session.Session()
model_name = 'clarify-model'

model = estimator.create_model(name=model_name)
container_def = model.prepare_container_def()
session.create_model(model_name,
                     role,
                     container_def)

Using already existing model: clarify-model


'clarify-model'

Now that you have your model set up. Let's say hello to SageMaker Clarify!

In [20]:
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=session)

### Detecting Bias
SageMaker Clarify helps you detect possible pre- and post-training biases using a variety of metrics.
#### Writing DataConfig and ModelConfig
A `DataConfig` object communicates some basic information about data I/O to SageMaker Clarify. We specify where to find the input dataset, where to store the output, the target column (`label`), the header names, and the dataset type.

In [21]:
%store -r bucket
%store -r prefix
%store -r s3uri_train

In [22]:
col_names = pd.read_csv("/root//amazon-sagemaker-workshop/2-Modeling/config/training-dataset-with-header.csv").columns.to_list()
col_names

['Churn',
 'Account Length',
 'VMail Message',
 'Day Mins',
 'Day Calls',
 'Eve Mins',
 'Eve Calls',
 'Night Mins',
 'Night Calls',
 'Intl Mins',
 'Intl Calls',
 'CustServ Calls',
 'State_AK',
 'State_AL',
 'State_AR',
 'State_AZ',
 'State_CA',
 'State_CO',
 'State_CT',
 'State_DC',
 'State_DE',
 'State_FL',
 'State_GA',
 'State_HI',
 'State_IA',
 'State_ID',
 'State_IL',
 'State_IN',
 'State_KS',
 'State_KY',
 'State_LA',
 'State_MA',
 'State_MD',
 'State_ME',
 'State_MI',
 'State_MN',
 'State_MO',
 'State_MS',
 'State_MT',
 'State_NC',
 'State_ND',
 'State_NE',
 'State_NH',
 'State_NJ',
 'State_NM',
 'State_NV',
 'State_NY',
 'State_OH',
 'State_OK',
 'State_OR',
 'State_PA',
 'State_RI',
 'State_SC',
 'State_SD',
 'State_TN',
 'State_TX',
 'State_UT',
 'State_VA',
 'State_VT',
 'State_WA',
 'State_WI',
 'State_WV',
 'State_WY',
 'Area Code_408',
 'Area Code_415',
 'Area Code_510',
 "Int'l Plan_no",
 "Int'l Plan_yes",
 'VMail Plan_no',
 'VMail Plan_yes']

In [44]:
bias_report_output_path = f's3://{bucket}/{prefix}/clarify-bias-report'
bias_data_config = clarify.DataConfig(s3_data_input_path=s3uri_train,
                                      s3_output_path=bias_report_output_path,
                                      label='Churn',
                                      headers=col_names,
                                      dataset_type='text/csv')

A `ModelConfig` object communicates information about your trained model. To avoid additional traffic to your production models, SageMaker Clarify sets up and tears down a dedicated endpoint when processing.
* `instance_type` and `instance_count` specify your preferred instance type and instance count used to run your model on during SageMaker Clarify's processing. The testing dataset is small so a single standard instance is good enough to run this example. If your have a large complex dataset, you may want to use a better instance type to speed up, or add more instances to enable Spark parallelization.
* `accept_type` denotes the endpoint response payload format, and `content_type` denotes the payload format of request to the endpoint.

In [28]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

A `ModelPredictedLabelConfig` provides information on the format of your predictions. XGBoost model outputs probabilities of samples, so SageMaker Clarify invokes the endpoint then uses `probability_threshold` to convert the probability to binary labels for bias analysis. Prediction above the threshold is interpreted as label value `1` and below or equal as label value `0`.

In [46]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

#### Writing BiasConfig
SageMaker Clarify also needs information on what the sensitive columns (`facets`) are, what the sensitive features (`facet_values_or_threshold`) may be, and what the desirable outcomes are (`label_values_or_threshold`).
SageMaker Clarify can handle both categorical and continuous data for `facet_values_or_threshold` and for `label_values_or_threshold`. In this case we are using categorical data.

We specify this information in the `BiasConfig` API. Here that the positive outcome is being `churn`. We will group users if they don't have an International Plan (`Int'l Plan_no`), which could be associated to customers from diffent segments/behaviors. `group_name` is used to form subgroups for the measurement of Conditional Demographic Disparity in Labels (CDDL) and Conditional Demographic Disparity in Predicted Labels (CDDPL) with regards to Simpson’s paradox.

In [47]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[1],
                                facet_name="Int'l Plan_no",
                                facet_values_or_threshold=[100],
                                group_name="Account Length")

#### Pre-training Bias
Bias can be present in your data before any model training occurs. Inspecting your data for bias before training begins can help detect any data collection gaps, inform your feature engineering, and hep you understand what societal biases the data may reflect.

Computing pre-training bias metrics does not require a trained model.

#### Post-training Bias
Computing post-training bias metrics does require a trained model.

Unbiased training data (as determined by concepts of fairness measured by bias metric) may still result in biased model predictions after training. Whether this occurs depends on several factors including hyperparameter choices.


You can run these options separately with `run_pre_training_bias()` and `run_post_training_bias()` or at the same time with `run_bias()` as shown below.

In [48]:
clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all')


Job Name:  Clarify-Bias-2021-06-10-00-41-43-637
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-studio-us-east-2-309044321483/xgboost-churn/data/train/train.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-studio-us-east-2-309044321483/xgboost-churn/clarify-bias-report/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-studio-us-east-2-309044321483/xgboost-churn/clarify-bias-report', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
..........................I

#### Viewing the Bias Report
In Studio, you can view the results under the experiments tab:

<img src="./media/bias_report_experiments.png" width="30%">

Right click on the unassigned trials:

<img src="./media/bias_report_right_click.png" width="30%">

Each bias metric has detailed explanations with examples that you can explore.

<img src="./media/bias_report_table.png">


If you're not a Studio user yet, you can access the bias report in pdf, html and ipynb formats in the following S3 bucket:

In [49]:
bias_report_output_path

's3://sagemaker-studio-us-east-2-309044321483/xgboost-churn/clarify-bias-report'

### Explaining Predictions
There are expanding business needs and legislative regulations that require explanations of _why_ a model made the decision it did. SageMaker Clarify uses SHAP to explain the contribution that each input feature makes to the final decision.

Kernel SHAP algorithm requires a baseline (also known as background dataset). Baseline dataset type shall be the same as `dataset_type` of `DataConfig`, and baseline samples shall only include features. By definition, `baseline` should either be a S3 URI to the baseline dataset file, or an in-place list of samples. In this case we chose the latter, and put the first sample of the test dataset to the list.

In [23]:
cn = col_names.copy()
cn.remove("Churn")

In [24]:
test_sample = df.iloc[0].values.tolist()
pd.DataFrame(columns=cn, data=[test_sample])

,Account Length,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,Intl Mins,Intl Calls,...,State_WI,State_WV,State_WY,Area Code_408,Area Code_415,Area Code_510,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes
0,186.0,0.0,137.8,97.0,187.7,118.0,146.4,85.0,8.7,6.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [26]:
shap_config = clarify.SHAPConfig(baseline=[test_sample],
                                 num_samples=15,
                                 agg_method='mean_abs',
                                 save_local_shap_values=False)

explainability_output_path = f's3://{bucket}/{prefix}/clarify-explainability'
explainability_data_config = clarify.DataConfig(s3_data_input_path=s3uri_train,
                                s3_output_path=explainability_output_path,
                                label='Churn',
                                headers=col_names,
                                dataset_type='text/csv')

In [ ]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config)


Job Name:  Clarify-Explainability-2021-06-10-01-18-19-063
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-studio-us-east-2-309044321483/xgboost-churn/data/train/train.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-studio-us-east-2-309044321483/xgboost-churn/clarify-explainability/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-studio-us-east-2-309044321483/xgboost-churn/clarify-explainability', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
...........

#### Viewing the Explainability Report
As with the bias report, you can view the explainability report in Studio under the experiments tab


<img src="./recordings/explainability_detail.gif">

The Model Insights tab contains direct links to the report and model insights.

If you're not a Studio user yet, as with the Bias Report, you can access this report at the following S3 bucket.

In [ ]:
explainability_output_path